In [1]:
import numpy as np
from scipy.sparse.linalg import eigsh
import numpy.linalg as la

In [3]:

def chebyshev_filter(x, m, a, b, a0, A,):
    e = (b-a)/2
    c = (b+a)/2
    s = e/(a0-c)
    s1=s
    y = (np.dot(A,x) - c*x)*(s1/e)
    for i in range(1,m):
        s_new = 1/((2/s1)-s)
        y_new = (np.dot(A,y) - c*y)*(2*s_new/e)-((s*s_new)*x)
        x=y
        y=y_new
        s=s_new
    return y



def orthonormalize(V):
    Q, _ = np.linalg.qr(V)
    return Q


def chebyshev_filtered_subspace(A, k=3, subspace_dim=20, 
                                degree=20, num_cycles=10):
    n = A.shape[0]

    
    # 1. Rough spectral bounds using Lanczos
    eig_max = eigsh(A, k=1, which='LA')[0][0]
    eig_min = eigsh(A, k=1, which='SA')[0][0]

    # shift to target smallest eigenvalues: filter below midpoint
    a = eig_min
    b = eig_max

    # 2. Initialize random subspace
    V = np.random.randn(n, subspace_dim)
    V = orthonormalize(V)
    for cycle in range(num_cycles):
        # 3. Apply Chebyshev filter
        #V = chebyshev_filter(V, A, a, b, degree)
        V = chebyshev_filter(V, degree, a, b,eig_min, A)

        # 4. Re-orthonormalize
        V = orthonormalize(V)

        # 5. Rayleigh-Ritz projection
        T = V.T @ A @ V
        evals, evecs = np.linalg.eigh(T)

        # 6. Update subspace with Ritz vectors
        V = V @ evecs

    # Extract k smallest Ritz vectors
    idx = np.argsort(evals)
    return evals[idx][:k], V[:, idx[:k]]


# ---------------------
# Example Usage
# ---------------------
if __name__ == "__main__":
    np.random.seed(0)
    n = 200
    A = np.random.randn(n, n)
    A = (A + A.T) / 2   # make symmetric

    evals, evecs = chebyshev_filtered_subspace(
        A,
        k=3,
        subspace_dim=20,
        degree=50,
        num_cycles=200
    )

    print("Computed Eigenvalues:", evals)
    print(eigsh(A))

Computed Eigenvalues: [-19.35720995 -16.96000701 -15.65506421]
(array([-19.37429901, -19.01347076, -18.47113565,  18.48408624,
        18.67296657,  19.2172738 ]), array([[ 0.04254426, -0.04595755, -0.04773205, -0.05262346,  0.1024978 ,
        -0.01851162],
       [-0.09025721,  0.0954689 ,  0.10546289,  0.05639254,  0.06076348,
        -0.06011502],
       [ 0.03405594, -0.06487484, -0.00181169,  0.05022531,  0.07478081,
         0.09283309],
       ...,
       [-0.05383828,  0.00698584,  0.01975547, -0.03597016, -0.07914974,
         0.06938719],
       [ 0.00248204,  0.03007355,  0.02492027,  0.0860022 , -0.0577709 ,
        -0.08746879],
       [ 0.06662727,  0.04027884,  0.03450743, -0.00086736, -0.1551787 ,
        -0.08232582]], shape=(200, 6)))


In [33]:
def chebyshev_filter(x, m, a, b, a0, A,):
    e = (b-a)/2
    c = (b+a)/2
    s = e/(a0-c)
    s1=s
    y = (np.dot(A,x) - c*x)*(s1/e)
    for i in range(1,m):
        s_new = 1/((2/s1)-s)
        y_new = (np.dot(A,y) - c*y)*(2*s_new/e)-((s*s_new)*x)
        x=y
        y=y_new
        s=s_new
    return y



def orthonormalize(V):
    Q, _ = np.linalg.qr(V)
    return Q


def chebyshev_filtered_subspace(A, k=3, subspace_dim=20, 
                                degree=20, tol=1e-8):
    n = A.shape[0]

    
    # 1. Rough spectral bounds using Lanczos
    eig_max = eigsh(A, k=1, which='LA')[0][0]
    eig_min = eigsh(A, k=1, which='SA')[0][0]

    # shift to target smallest eigenvalues: filter below midpoint
    a = eig_min
    b = eig_max

    # 2. Initialize random subspace
    V = np.random.randn(n, subspace_dim)
    V = orthonormalize(V)

    iter = 0
    evals_old=np.zeros(k)
    r = 0
    while iter == 0 or r > tol:
        iter += 1
        
        # 3. Apply Chebyshev filter
        #V = chebyshev_filter(V, A, a, b, degree)
        V = chebyshev_filter(V, degree, a, b,eig_min, A)

        # 4. Re-orthonormalize
        V = orthonormalize(V)

        # 5. Rayleigh-Ritz projection
        T = V.T @ A @ V
        evals, evecs = np.linalg.eigh(T)

        # 6. Update subspace with Ritz vectors
        V = V @ evecs

        idx = np.argsort(evals)
        r = la.norm(evals[idx][:k]-evals_old,1)
        evals_old = evals[idx][:k]

    # Extract k smallest Ritz vectors
    idx = np.argsort(evals)
    return evals[idx][:k], V[:, idx[:k]]

In [37]:
# ---------------------
# Example Usage
# ---------------------
np.random.seed(0)
n = 200
A = np.random.randn(n, n)
A = (A + A.T) / 2   # make symmetric

evals, evecs = chebyshev_filtered_subspace(
    A,
    k=2,
    subspace_dim=20,
    degree=50,
    #num_cycles=2000
)

print("Computed Eigenvalues:", evals)
print(eigsh(A))

Computed Eigenvalues: [-19.37429899 -17.00184331]
(array([-19.37429901, -19.01347076, -18.47113565,  18.48408624,
        18.67296657,  19.2172738 ]), array([[-0.04254426,  0.04595755, -0.04773205, -0.05262346,  0.1024978 ,
         0.01851162],
       [ 0.09025721, -0.0954689 ,  0.10546289,  0.05639254,  0.06076348,
         0.06011502],
       [-0.03405594,  0.06487484, -0.00181169,  0.05022531,  0.07478081,
        -0.09283309],
       ...,
       [ 0.05383828, -0.00698584,  0.01975547, -0.03597016, -0.07914974,
        -0.06938719],
       [-0.00248204, -0.03007355,  0.02492027,  0.0860022 , -0.0577709 ,
         0.08746879],
       [-0.06662727, -0.04027884,  0.03450743, -0.00086736, -0.1551787 ,
         0.08232582]], shape=(200, 6)))


In [40]:
import chebDev
chebDev.cheb_dav(A,3)[0]



array([-19.37429901, -19.01347076, -18.47113565])

In [ ]:
verbose = 0

def chebyshev_filter(x, m, a, b, a0, A, matvec, matmat, vecvec):
    e = (b-a)/2
    c = (b+a)/2
    s = e/(a0-c)
    s1=s
    y = (np.dot(A,x) - c*x)*(s1/e)
    matvec += 1
    for i in range(1,m):
        s_new = 1/((2/s1)-s)
        y_new = (np.dot(A,y) - c*y)*(2*s_new/e)-((s*s_new)*x)
        matvec += 1
        x=y
        y=y_new
        s=s_new
    return y, matvec, matmat, vecvec

def swapIfNeeded(eigVal, eigVec, matvec, matmat, vecvec):
    mu = eigVal[-1]
    i = eigVal.shape[0]-1
    maxI = i
    while mu < eigVal[i-1]:
        i -= 1
    sortedIndex=list(range(i))+[maxI]+list(range(i,maxI))
    eigVecSortedIndex = sortedIndex
    if eigVec.shape[0]>eigVal.shape[0]:
        eigVecSortedIndex = np.append(sortedIndex,np.array(range(len(eigVal), eigVec.shape[1])))
    return eigVal[sortedIndex], eigVec[:,eigVecSortedIndex], i!=maxI, matvec, matmat, vecvec

def orthonormalizeDGKS(v, M, matvec, matmat, vecvec):
    vnorm = la.norm(v)
    v = v/vnorm
    vnew_norm = None
    eta = 1/np.sqrt(2)
    rndVecIter = 0
    dgksiter = 0
    tmpnorm = eta*vnorm
    while vnew_norm is None or vnew_norm < tmpnorm:
        v_tmp = M.transpose().dot(v)
        matvec += 1
        tmpnorm = la.norm(v_tmp)
        v_new = v - M.dot(v_tmp)
        matvec += 1
        vnew_norm = la.norm(v_new)
        v_new = v_new/vnew_norm
        dgksiter += 1
        if dgksiter > 3:
            print("max dgks iter reached:", vnew_norm, eta*vnorm)
            if rndVecIter < 3:
                rndVecIter += 1
                v =  np.matrix(np.random.randn(v.shape[0])).transpose()
            else:
                raise Exception("DGKS could not orthornormalize after 3 iterations and 3 random vectors")
        else:
            v = v_new
    return v_new, matvec, matmat, vecvec


def lanczos_upperb(A, k, matvec, matmat, vecvec):
    dim = A.shape[0]
    if k == -1:
        k=dim
    T = np.matrix(np.zeros((k, k), dtype=float))

    v_arr = []
    v = np.matrix(np.random.randn(A.shape[0])).transpose()
    v = v/la.norm(v)
    w_prime = A.dot(v)
    matvec += 1
    a = v.transpose().dot(w_prime)[0,0]
    vecvec += 1
    w_new = w_prime - a*v
    v_arr.append(v)
    T[0,0]=a
    for i in range(1,k):
        b=la.norm(w_new)
        if b!=0:
            v_new = w_new/b
            v_arr.append(v_new)
        else:
            v_arr = generate_linearly_independent_unit_vector(v_arr)
            v_new = np.matrix(v_arr[-1]).transpose()
        w_prime = A.dot(v_new)
        matvec += 1
        a = v_new.transpose().dot(w_prime)[0,0]
        w_new = w_prime - a*v_new - b*v
        v=v_new
        T[i,i] = a
        T[i-1, i] = b
        T[i,i-1] = b
    max_eig = max(la.eig(T[:i+1, :i+1])[0])
    if b < 0.01:
        return max_eig+b*10, matvec, matmat, vecvec
    elif b < 0.1:
        return max_eig+b*5, matvec, matmat, vecvec
    else:
        return max_eig+b, matvec, matmat, vecvec
    
    
def getUpperBound(A, matvec, matmat, vecvec):
    nrm_1 = la.norm(A,1)
    l_upperb, matvec, matmat, vecvec = lanczos_upperb(A, 4, matvec, matmat, vecvec)
    return min(nrm_1, l_upperb), matvec, matmat, vecvec
    
def cheb_dav(A, kwant, x = None, m=10, kkeep=None, maxdim=None, tol=1e-10, maxIter = 1e3):
    matvec = 0
    matmat = 0
    vecvec = 0
    if x is None:
        x = np.matrix(np.random.randn(A.shape[0])).transpose()
    if kkeep is None:
        kkeep = kwant*2
    if maxdim is None:
        maxdim=kwant*4
        
    eigs = np.array([])
    x = x/la.norm(x)
    V = np.matrix(x)
    w = A.dot(x)
    matvec += 1
    W = np.matrix(w)
    mu = x.transpose().dot(w)[0,0]
    vecvec += 1
    H = np.matrix(mu)
    
    r = w - mu*x
    if la.norm(r,2) <= tol:
        eigs = np.append(eigs, mu)
        kc = 1
        H = np.empty((0,0))
    else:
        kc=0
    upperb, matvec, matmat, vecvec = getUpperBound(A, matvec, matmat, vecvec)
    lowerb = (upperb + mu)/2
    a0 = lowerb
    ksub = 0
    itercount=0
    while itercount < maxIter:
        if verbose > 1:
            print('Upperbound', upperb)
            print('Lowerbound', lowerb)
        t, matvec, matmat, vecvec = chebyshev_filter(x,m,lowerb, upperb, a0, A, matvec, matmat, vecvec)
        v_knext, matvec, matmat, vecvec = orthonormalizeDGKS(t,V[:,0:ksub+1], matvec, matmat, vecvec)
        V = np.column_stack([V, v_knext])
        kold = ksub
        ksub += 1
        w_knext = A.dot(v_knext)
        matvec += 1
        W = np.column_stack([W,w_knext])
        if verbose > 1:
            print('V', V.shape,'W', W.shape,'H', H.shape)
            print('kc:',kc, ',    ksub:',ksub, ',     kold:',kold)        
        #H=np.row_stack([H[:ksub-kc,:ksub-kc],np.zeros(ksub-kc)])
        H=np.row_stack([H,np.zeros(kold+1)])
        if kc>0:
            H=np.column_stack([H,np.row_stack([np.zeros((kc,1)), V[:,kc:ksub+1].transpose().dot(w_knext)])])
            matvec += 1
        else:
            H=np.column_stack([H,V[:,kc:ksub+1].transpose().dot(w_knext)])
            matvec += 1
        for i in range(H.shape[0]-1):
            H[-1,i]=H[i,-1]
        D,Y = la.eigh(H[kc:ksub+1, kc:ksub+1])
        if verbose > 1:
            print('Y',Y)
            print('D',D)
        mu = D[0]
        if ksub >= maxdim:
            if verbose > 0:
                print("INNER RESTART")
            ksub = kc+kkeep
            #H = H[:ksub, :ksub]
            #V = V[:,:ksub]
        if verbose > 0:
            print('kc',kc, '      kold',kold,'         ksub',ksub)
        if kc > 0:
            V = np.column_stack([V[:,:kc], V[:,kc:kold+2].dot(Y[:,:ksub-kc+1])])
            W = np.column_stack([W[:,:kc], W[:,kc:kold+2].dot(Y[:,:ksub-kc+1])])
            matmat += 2
        else:
            V = np.column_stack([V[:,:kc], V[:,kc:kold+2].dot(Y[:,:ksub-kc+1])])
            W = np.column_stack([W[:,:kc], W[:,kc:kold+2].dot(Y[:,:ksub-kc+1])])
            matmat += 2
        if kc>0:
            H = np.column_stack([np.row_stack([H[:kc,:kc],np.zeros((ksub+1-kc,kc))]),np.row_stack([np.zeros((kc,ksub+1-kc)),np.diag(D[:ksub-kc+1])])])
        else:
            H = np.diag(D[:ksub+1])
        r = W[:,kc]-mu*V[:,kc]
        if verbose > 0:
            print('2-norm r',la.norm(r,2))

        itercount +=1
        for conv_i in range(len(D)):
            if verbose > 1:
                print(f'convergenceCheck: norm(r): {la.norm(r,2)}, {tol*max(D[conv_i:])}, V {V.shape}, W {W.shape}, H {H.shape}, norm(H) {la.norm(H,2)}') 
            if la.norm(r,2) <= tol*max(D[conv_i:]):
                kc += 1
                mu = D[conv_i]
                eigs = np.append(eigs,mu)
                if verbose > 0: 
                    print(f"CONVERGED: #{eigs.shape[0]}, in {itercount} steps:",mu)
                ## SWAP TEST and set swap=True if swap happens
                swap = False
                if kc > 1:
                    eigs, V, swap, matmat, vecvec, matvec = swapIfNeeded(eigs, V, matmat, vecvec, matvec)
                    
                if kc >= kwant and not swap:
                    return eigs, V[:,:kc+1], matmat, vecvec, matvec
                mu = D[conv_i+1]
                if kc < V.shape[1]:
                    r = W[:,kc]-mu*V[:,kc]
            else:
                break
        lowerb = np.median(D[:ksub])
        if a0 > min(D):
            a0 = min(D)
        x = V[:,kc]


In [ ]:
cheb_dav(A,3)